In [ ]:
from importlib import reload
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Add the project root to the path
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, project_root)

import pacing_auction.data as data
import pacing_auction.auction as auction
import pacing_auction.elimination as elimination
import pacing_auction.generator as generator
reload(data)
reload(elimination)
reload(auction)
reload(generator)

sns.set_theme("paper")

In [ ]:
df1 = pd.read_csv("../results/v2.1/data.csv")
df1_matrices = np.load("../results/v2.1/auction_matrices.npz", allow_pickle=True)

print(f"Matrices: {df1_matrices.files}")
vs1 = [np.array(v) for v in df1_matrices["v_matrices"]]
bs1 = [np.array(b) for b in df1_matrices["b_vectors"]]
alpha_qs1 = [np.array(a) for a in df1_matrices["alpha_q_vectors"]]
print(f"vs1: {len(vs1)}")
print(f"bs1: {len(bs1)}")
print(f"alpha_qs: {len(alpha_qs1)}")

revenues1 = []
liquid_welfare1 = []
social_welfare1 = []
for v, b, alpha_q in zip(vs1, bs1, alpha_qs1):
    n, m = v.shape
    a = auction.Auction(n, m)
    a.v = v
    a.b = b
    a.alpha_q = alpha_q
    x, p = a.auction()
    revenue = a.revenue(x, p)
    revenues1.append(revenue)

    liquid_welfare = a.liquid_welfare(x, p)
    liquid_welfare1.append(liquid_welfare)

    social_welfare = a.social_welfare(x, p)
    social_welfare1.append(social_welfare)

df1["revenue"] = revenues1
df1["liquid_welfare"] = liquid_welfare1
df1["social_welfare"] = social_welfare1

In [ ]:
df2 = pd.read_csv("../results/v2.2/data.csv")
df2_matrices = np.load("../results/v2.2/auction_matrices.npz", allow_pickle=True)

print(f"Matrices: {df2_matrices.files}")
vs2 = [np.array(v) for v in df2_matrices["v_matrices"]]
bs2 = [np.array(b) for b in df2_matrices["b_vectors"]]
alpha_qs2 = [np.array(a) for a in df2_matrices["alpha_q_vectors"]]
print(f"vs1: {len(vs2)}")
print(f"bs1: {len(bs2)}")
print(f"alpha_qs: {len(alpha_qs2)}")

revenues2 = []
liquid_welfare2 = []
social_welfare2 = []
for v, b, alpha_q in zip(vs2, bs2, alpha_qs2):
    n, m = v.shape
    a = auction.Auction(n, m)
    a.v = v
    a.b = b
    a.alpha_q = alpha_q
    x, p = a.auction()
    revenue = a.revenue(x, p)
    revenues2.append(revenue)

    liquid_welfare = a.liquid_welfare(x, p)
    liquid_welfare2.append(liquid_welfare)

    social_welfare = a.social_welfare(x, p)
    social_welfare2.append(social_welfare)

df2["revenue"] = revenues2
df2["liquid_welfare"] = liquid_welfare2
df2["social_welfare"] = social_welfare2

In [ ]:
df = pd.concat([df1, df2])
df['time_per_iteration'] = df['runtime'] / df['iterations']
df

In [ ]:
print(f"Timeouts: {df[df["timeout"]].shape[0]}")
print(f"Errors: {df[df["error"]].shape[0]}")

In [ ]:
df_time_per_iteration_pivot = df.pivot_table(
    index='n', columns='m', values='time_per_iteration', aggfunc='median'
)

plt.figure(figsize=(10, 8))
sns.heatmap(df_time_per_iteration_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Time per Iteration (s)'})
plt.title('Heatmap of Time per Iteration by n and m (Complete)')
plt.xlabel('m')
plt.ylabel('n')
plt.show()

In [ ]:
complete = df[df["generator"] == "complete"]
sampled = df[df["generator"] == "sampled"]
correlated = df[df["generator"] == "correlated"]

elim_all = df[df["elim_strategy"] == "all"]
elim_subsequent = df[df["elim_strategy"] == "subsequent"]
elim_current = df[df["elim_strategy"] == "current"]

print(complete.shape)
print(sampled.shape)
print(correlated.shape)

print(elim_all.shape)
print(elim_subsequent.shape)
print(elim_current.shape)

In [ ]:
complete_pivot = complete.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
sampled_pivot = sampled.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)
correlated_pivot = correlated.pivot_table(
    index='n', columns='m', values='result_type', aggfunc=lambda x: (x == 'PNE').mean()
)

# Generate the heatmaps
plt.figure(figsize=(10, 8))
sns.heatmap(complete_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Percentage of PNE'})
plt.title('Heatmap of Percentage of PNE by n and m (Complete)')
plt.xlabel('m')
plt.ylabel('n')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(sampled_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Percentage of PNE'})
plt.title('Heatmap of Percentage of PNE by n and m (Sampled)')
plt.xlabel('m')
plt.ylabel('n')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(correlated_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Percentage of PNE'})
plt.title('Heatmap of Percentage of PNE by n and m (Correlated)')
plt.xlabel('m')
plt.ylabel('n')
plt.show()

In [ ]:
# Create pivot tables for the number of iterations
complete_iterations_pivot = complete.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)
sampled_iterations_pivot = sampled.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)
correlated_iterations_pivot = correlated.pivot_table(
    index='n', columns='m', values='iterations', aggfunc='median'
)

# Generate the heatmaps for iterations
plt.figure(figsize=(10, 8))
sns.heatmap(complete_iterations_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Median Iterations'})
plt.title('Heatmap of Median Iterations by n and m (Complete)')
plt.xlabel('m')
plt.ylabel('n')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(sampled_iterations_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Median Iterations'})
plt.title('Heatmap of Median Iterations by n and m (Sampled)')
plt.xlabel('m')
plt.ylabel('n')
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(correlated_iterations_pivot, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Median Iterations'})
plt.title('Heatmap of Median Iterations by n and m (Correlated)')
plt.xlabel('m')
plt.ylabel('n')
plt.show()


In [ ]:
pnes = df[df['result_type'] == 'PNE']
cycles = df[df['result_type'] == 'Cycle']
print(f"PNEs: {pnes.shape[0]}, {round(pnes.shape[0] * 100 / df.shape[0], 2)}%")

# plt.figure()
# sns.histplot(pnes[pnes['iterations'] > 3]['iterations'], bins=100, kde=True) # type: ignore

# plt.figure()
# sns.histplot(cycles[cycles["iterations"] < 1000]['iterations'], bins=100, kde=True) # type: ignore



In [ ]:
pnes_complete = pnes[pnes['generator'] == 'complete']
cycles_complete = cycles[cycles['generator'] == 'complete']
print(f"PNEs (Complete): {pnes_complete.shape[0]}, {round(pnes_complete.shape[0] * 100 / complete.shape[0], 2)}%")

# plt.figure()
# sns.histplot(pnes_complete[pnes_complete['iterations'] > 3]['iterations'], bins=100, kde=True) # type: ignore

# plt.figure()
# sns.histplot(cycles_complete[cycles_complete["iterations"] < 1000]['iterations'], bins=100, kde=True) # type: ignore

In [ ]:
pnes_sampled = pnes[pnes['generator'] == 'sampled']
cycles_sampled = cycles[cycles['generator'] == 'sampled']
print(f"PNEs (Sampled): {pnes_sampled.shape[0]}, {round(pnes_sampled.shape[0] * 100 / sampled.shape[0], 2)}%")

# plt.figure()
# sns.histplot(pnes_sampled[pnes_sampled['iterations'] > 3]['iterations'], bins=100, kde=True) # type: ignore

# plt.figure()
# sns.histplot(cycles_sampled[cycles_sampled["iterations"] < 1000]['iterations'], bins=100, kde=True) # type: ignore

In [ ]:
pnes_correlated = pnes[pnes['generator'] == 'correlated']
cycles_correlated = cycles[cycles['generator'] == 'correlated']
print(f"PNEs (Correlated): {pnes_correlated.shape[0]}, {round(pnes_correlated.shape[0] * 100 / correlated.shape[0], 2)}%")

plt.figure()
sns.histplot(pnes_correlated['iterations'], bins=1000, kde=True) # type: ignore

plt.figure()
sns.histplot(cycles_correlated['iterations'], bins=1000, kde=True) # type: ignore

In [ ]:
from matplotlib.ticker import FuncFormatter
# Your existing code up until the plotting part...

plt.figure(figsize=(12, 8))

# Calculate percentage of PNEs for each sigma-delta combination
sigma_delta_pne = []
for (sigma, delta), df_group in correlated.groupby(["sigma", "delta"]):
    pnes_group = df_group[df_group['result_type'] == 'PNE']
    pne_percentage = (pnes_group.shape[0] / df_group.shape[0]) * 100

    sigma_delta_pne.append({'sigma': sigma, 'delta': delta, 'pne_percentage': pne_percentage})

# Convert to DataFrame
df_sigma_delta = pd.DataFrame(sigma_delta_pne)


# Get unique sigma values
sigma_values = sorted(df_sigma_delta['sigma'].unique())

# Plot a line for each sigma value
for sigma in sigma_values:
    sigma_data = df_sigma_delta[df_sigma_delta['sigma'] == sigma]
    sigma_data = sigma_data.sort_values('delta')
    plt.plot(sigma_data['delta'], sigma_data['pne_percentage'], marker='o', linewidth=2, label=f'σ = {sigma}')

# Format y-axis to show percentage
def percentage_formatter(x, p):
    return f'{x:.0f}%'

plt.gca().yaxis.set_major_formatter(FuncFormatter(percentage_formatter))

plt.title('Percentage of PNEs by Delta (δ) for Each Sigma (σ) Value', fontsize=16)
plt.xlabel('Delta (δ)', fontsize=14)
plt.ylabel('Percentage of PNEs (%)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(title='Sigma Values', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
display(pnes.describe())
display(cycles.describe())